In [1]:
import pandas as pd


# Import CSV of fitbit weartime using heartrates
df = pd.read_csv('fitbitWearTimeViaHR_merged.csv')

In [2]:
df

,Id,Day,TotalMinutesWearTime,PercentageWearTime
0,53211,8/18/2024,0,0.000000
1,53211,8/19/2024,0,0.000000
2,53211,8/20/2024,0,0.000000
3,53211,8/21/2024,0,0.000000
4,53211,8/22/2024,0,0.000000
...,...,...,...,...
347,53917,9/14/2024,1440,100.000000
348,53917,9/15/2024,1423,98.819444
349,53917,9/16/2024,1440,100.000000
350,53917,9/17/2024,1440,100.000000


In [3]:
# Return wear time and sync data stats!

# 5/7 days worn (Consecutively?) 
# Percent daily?

# Return Ids and personalized message? (Can be good or bad messages)
# Personalized weartime? Days and mean weartime.
# Didn't wear at all
# Sync Events >3 days of no weartime?


# Reminder-return stats and binary good vs bad message

# We will keep track of average documented weartime, total number of days worn, number of syncs, and last time synced.
reminder_wearDf = df.groupby('Id').agg(
    AverageWearTime = ('PercentageWearTime', 'mean'),
    DaysWorn = ('TotalMinutesWearTime', lambda x: (x > 0).sum())
    
).reset_index()
reminder_wearDf

,Id,AverageWearTime,DaysWorn
0,53211,63.621962,23
1,53212,61.584201,23
2,53411,17.860243,8
3,53412,27.992622,20
4,53911,64.433594,22
5,53912,54.092882,21
6,53913,61.846788,22
7,53914,64.891493,22
8,53915,63.287760,22
9,53916,33.719618,21


In [4]:
# Read sync data
syncDf = pd.read_csv('syncEvents_merged.csv')

syncDf

,Id,DateTime,SyncDateUTC,Provider,DeviceName
0,53211,8/27/2024 11:47:10 AM,8/27/2024 6:47:10 PM,Fitbit,Inspire 3
1,53211,8/27/2024 11:51:37 AM,8/27/2024 6:51:37 PM,Fitbit,Inspire 3
2,53211,8/27/2024 11:51:46 AM,8/27/2024 6:51:46 PM,Fitbit,Inspire 3
3,53211,8/27/2024 11:53:42 AM,8/27/2024 6:53:42 PM,Fitbit,Inspire 3
4,53211,8/27/2024 11:55:28 AM,8/27/2024 6:55:28 PM,Fitbit,Inspire 3
...,...,...,...,...,...
14292,53917,9/18/2024 9:31:15 PM,9/19/2024 2:31:15 AM,Fitbit,Inspire 3
14293,53917,9/18/2024 9:46:20 PM,9/19/2024 2:46:20 AM,Fitbit,Inspire 3
14294,53917,9/18/2024 10:01:26 PM,9/19/2024 3:01:26 AM,Fitbit,Inspire 3
14295,53917,9/18/2024 10:16:32 PM,9/19/2024 3:16:32 AM,Fitbit,Inspire 3


In [5]:
# Function to determine days synced
def uniqueDays(group):
    days = 0
    previousDay = None

    for dateTime in group:
        currentDay = str(dateTime)
        if(previousDay is None or currentDay[:4] != previousDay[:4]):
            days += 1
        previousDay = currentDay
    return days

# Sort last day synced and total number of syncs
reminder_syncDf = syncDf.groupby('Id').agg(
    LastDaySynced = ('DateTime', 'last'),
    DaysSynced = ('DateTime', uniqueDays),
    TotalSyncs = ('DateTime', 'count')
).reset_index()

reminder_syncDf

,Id,LastDaySynced,DaysSynced,TotalSyncs
0,53211,9/18/2024 10:24:07 PM,23,2045
1,53212,9/18/2024 3:15:31 PM,22,1927
2,53411,9/18/2024 10:32:13 PM,7,258
3,53412,9/18/2024 10:39:19 PM,12,694
4,53911,9/18/2024 1:25:33 PM,22,173
5,53912,9/17/2024 10:39:39 PM,17,130
6,53913,9/18/2024 10:32:44 PM,22,1928
7,53914,9/18/2024 3:01:11 PM,19,1435
8,53915,9/18/2024 10:22:52 PM,22,1901
9,53916,9/18/2024 10:27:18 PM,22,1841


In [6]:
# Combine Weartime and Sync dataframes
reminder_df = pd.merge(reminder_wearDf, reminder_syncDf, on = 'Id', how = 'outer')
reminder_df

,Id,AverageWearTime,DaysWorn,LastDaySynced,DaysSynced,TotalSyncs
0,53211,63.621962,23,9/18/2024 10:24:07 PM,23,2045
1,53212,61.584201,23,9/18/2024 3:15:31 PM,22,1927
2,53411,17.860243,8,9/18/2024 10:32:13 PM,7,258
3,53412,27.992622,20,9/18/2024 10:39:19 PM,12,694
4,53911,64.433594,22,9/18/2024 1:25:33 PM,22,173
5,53912,54.092882,21,9/17/2024 10:39:39 PM,17,130
6,53913,61.846788,22,9/18/2024 10:32:44 PM,22,1928
7,53914,64.891493,22,9/18/2024 3:01:11 PM,19,1435
8,53915,63.287760,22,9/18/2024 10:22:52 PM,22,1901
9,53916,33.719618,21,9/18/2024 10:27:18 PM,22,1841


In [9]:
# Determine what messages people should receive?
# Set threshold values for consistency
min_consecutive_days_worn = 3
min_average_wear_time = 50.0
max_days_not_synced = 3

# Calculate consecutive days worn
def calc_consecutive_days_worn(df):
    df['Day'] = pd.to_datetime(df['Day'])
    df['ConsecutiveDaysWorn'] = (df['Day'] - df['Day'].shift(1)).dt.days.ne(1).cumsum()
    return df.groupby('ConsecutiveDaysWorn').size().max()

reminder_df['ConsecutiveDaysWorn'] = df.groupby('Id').apply(calc_consecutive_days_worn).reset_index(drop=True)

# Calculate consecutive days synced
def calc_consecutive_days_synced(df):
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df['ConsecutiveDaysSynced'] = (df['DateTime'].dt.date - df['DateTime'].dt.date.shift(1)).dt.days.ne(1).cumsum()
    return df.groupby('ConsecutiveDaysSynced').size().max()

reminder_df['ConsecutiveDaysSynced'] = syncDf.groupby('Id').apply(calc_consecutive_days_synced).reset_index(drop=True)

# Generate personalized reminders
reminder_df['Reminder'] = ''

for index, row in reminder_df.iterrows():
    reminder = ''
    
    if row['ConsecutiveDaysWorn'] < min_consecutive_days_worn:
        reminder += f"You have only worn the device for {row['ConsecutiveDaysWorn']} consecutive days in the last month. Please aim to wear it for at least {min_consecutive_days_worn} consecutive days! "
    
    if row['AverageWearTime'] < min_average_wear_time:
        reminder += f"In the last month, your average wear time was {row['AverageWearTime']:.2f}%. Next month, please remember to wear it more consistently to further help our reseach. Let's aim for {min_consecutive_days_worn} consecutive days!"
    
    last_sync_date = pd.to_datetime(row['LastDaySynced'])
    days_since_last_sync = (pd.Timestamp.now() - last_sync_date).days
    
    if days_since_last_sync > max_days_not_synced:
        reminder += f"It has been {days_since_last_sync} days since your last sync. Please remember to sync your device regularly. "
    
    if row['ConsecutiveDaysSynced'] < min_consecutive_days_worn:
        reminder += f"You have only synced the device for {row['ConsecutiveDaysSynced']} consecutive days. Please aim to sync it for at least {min_consecutive_days_worn} consecutive days. "
    
    if reminder == '':
        reminder = "Great job! You have been wearing and syncing your device consistently. Keep up the good work!"
    
    reminder_df.at[index, 'Reminder'] = reminder.strip()

# Save the reminders to a CSV file
reminder_df.to_csv('reminder_ids.csv', index=False)